# Gravity Model Walkthrough

Based on code developed by Sadra Daneshvar (2023): https://github.com/SadraDaneshvar/Gravity_Model/blob/main/README.md

### Input
Input for this notebook: CSV list of production-attraction zones. It must have the following columns:
- zone_id
- x_coord (longitude, e.g., -123.4567)
- y_coord (latitude, e.g., 40.4401)
- production
- attraction
- zone_type (I for internal or E for external)

### Output
Output from this notebook: CSV trip tables for both production-attractions and origin-destinations

This code sets II (internal-internal), IE, and EI trips. It does not set EE (external-external) trips.

# Imports and Functions

In [ ]:
import numpy as np  # Import numpy for numerical operations
import pandas as pd  # Import pandas for data manipulation
from math import radians, cos, sin, asin, sqrt
import csv

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in miles between two points 
    on the earth (specified in decimal degrees)
    Taken from the format_network helper tools
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3959.87433  # this is in miles
    return c * r

In [ ]:
def format_matrix(matrix, matrix_name):
    matrix_size = matrix.shape[0]  # Get the number of rows in the matrix
    # Create column names for the matrix
    column_names = [f"Zone {i}" for i in range(1, matrix_size + 1)]
    # Convert the matrix into a pandas DataFrame for pretty printing
    formatted_matrix = pd.DataFrame(
        matrix, columns=column_names, index=column_names
    )
    # Print the formatted matrix
    print(f"{matrix_name}:\n", formatted_matrix, "\n")

In [ ]:
def deterrence_function(cij, beta):
    return np.exp(-beta*cij)

In [ ]:
def gravity_model(
    O,  # Origin matrix
    D,  # Destination matrix
    cost_matrix,  # Cost matrix
    deterrence_matrix,  # Deterrence matrix
    error_threshold=0.01,  # Error threshold for stopping condition
    improvement_threshold=1e-4,  # Improvement threshold for stopping condition
):
    # Define a nested function to format and print matrices

    # Print the initial cost matrix and deterrence matrix
    format_matrix(cost_matrix, "Initial Cost Matrix")
    format_matrix(deterrence_matrix, "Deterrence Matrix")

    # Normalize O and D so their sums are equal
    sum_O = np.sum(O)  # Sum of all elements in O
    sum_D = np.sum(D)  # Sum of all elements in D
    # Adjust O or D if their sums are not equal
    if sum_O != sum_D:
        if sum_O < sum_D:
            correction_ratio = sum_D / sum_O  # Calculate correction ratio
            O = O * correction_ratio  # Adjust O by the correction ratio
        else:
            correction_ratio = sum_O / sum_D  # Calculate correction ratio
            D = D * correction_ratio  # Adjust D by the correction ratio

    n = len(O)  # Number of zones
    T = np.sum(O)  # Total number of trips

    # Initialize balancing factors Ai and Bj
    Ai = np.ones(n)  # Ai balancing factor, initially set to 1 for each zone
    Bj = np.ones(n)  # Bj balancing factor, initially set to 1 for each zone

    previous_error = np.inf  # Initialize previous error to infinity
    iteration_count = 0  # Initialize iteration count
    stop_reason = ""  # Initialize stop reason string

    # Iterative process
    while True:
        iteration_count += 1  # Increment iteration count

        # Update Ai balancing factors
        for i in range(n):
            Ai[i] = 1 / (np.sum(Bj * D * deterrence_matrix[i, :]) + 1e-9)

        # Update Bj balancing factors
        Bj_new = np.ones(n)  # Temporary array for new Bj values
        for j in range(n):
            Bj_new[j] = 1 / (np.sum(Ai * O * deterrence_matrix[:, j]) + 1e-9)

        # Calculate Tij matrix for the model
        Tij = np.outer(Ai * O, Bj_new * D) * deterrence_matrix

        # Calculate the error of the model
        error = (
            np.sum(np.abs(O - np.sum(Tij, axis=1)))
            + np.sum(np.abs(D - np.sum(Tij, axis=0)))
        ) / T

        # Calculate the change in error from the previous iteration
        error_change = abs(previous_error - error)

        # Check stopping conditions
        if error < error_threshold:
            stop_reason = "Error threshold met"  # Set stop reason
            break  # Break the loop if error threshold is met
        elif error_change < improvement_threshold:
            stop_reason = "Slow improvement"  # Set stop reason
            break  # Break the loop if improvement is slow

        previous_error = error  # Update the previous error
        Bj = Bj_new  # Update Bj with new values
    
    print(f"Number of Iterations: {iteration_count}")  # Print the number of iterations
    print(f"Stopping Condition: {stop_reason}")  # Print the stopping condition
    print(
        f"Error: {error*100:.3f}%"
    )  # Print the final error as a percentage with 3 decimal places

    return(Tij)

# Main Code

In [ ]:
# Set up input and output file paths
input_file = "C:\GitHub\RDR\Data\sample_gravity\PA.csv"
output_pa_file = "C:\GitHub\RDR\Data\sample_gravity\PAtrips.csv"
output_od_file = "C:\GitHub\RDR\Data\sample_gravity\ODtrips.csv"

In [ ]:
# Read in the network zones
df = pd.read_csv(input_file)
df.shape

In [ ]:
# Create distance matrix
# Calculate Haversine distance(coordinates, coordinates)
distances = np.zeros([df.shape[0], df.shape[0]])

In [ ]:
for i in df.index:
    for j in df.index:
        if(df['zone_type'][i] == 'E' and df['zone_type'][j] == 'E'):
            distances[i][j] = 9999
        else:
            distances[i][j] = haversine(float(df['x_coord'][i]), float(df['y_coord'][i]), float(df['x_coord'][j]), float(df['y_coord'][j]))


In [ ]:
print(distances)

In [ ]:
productions = df['production'].values
attractions = df['attraction'].values
zone_ids = df['zone_id'].values

In [ ]:
# Beta parameter for the deterrence function
beta = 0.1

# Deterrence matrix calculated using the deterrence function and the cost matrix
deterrence_matrix = deterrence_function(distances, beta)

# Set the error threshold for the stopping condition of the gravity model
error_threshold = 0.005

# Set the improvement threshold for the stopping condition of the gravity model
improvement_threshold = 0.000001

In [ ]:
Tij = gravity_model(productions, 
              attractions, 
              distances, 
              deterrence_matrix, 
              error_threshold, 
              improvement_threshold)

In [ ]:
Tij

In [ ]:
with open(output_pa_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['orig_node', 'dest_node', 'trips', 'orig_node_prod', 'dest_node_attr', 'distance'])
    i = 0
    for row in Tij:
        j = 0
        for element in row:
            csvwriter.writerow([zone_ids[i], zone_ids[j], element, productions[i], attractions[j], distances[i,j]])
            j = j+1
        i = i+1

In [ ]:
with open(output_od_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['orig_node', 'dest_node', 'trips', 'PAtrips', 'APtrips', 'orig_node_prod', 'dest_node_attr', 'distance'])
    i = 0
    for row in Tij:
        j = 0
        for element in row:
            csvwriter.writerow([zone_ids[i], zone_ids[j], Tij[i,j] + Tij[j,i], Tij[i,j], Tij[j,i], productions[i], attractions[j], distances[i,j]])
            j = j+1
        i = i+1